In [1]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [143]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [144]:
dataFile = gzip.open(path, 'r')

In [145]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [146]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [152]:
len(productReviews)

134476

In [153]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [154]:
productsReviewData.shape
productsReviewData.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [158]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]




In [159]:
rating = lambda x: 1 if x > 1.0 else 0

In [160]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = (reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall'))
unfavorableReviews

,reviewText,helpful,overall,rating
56,These burn out really fast. I buy them becaus...,"[0, 0]",1.0,0
85276,My unit is supplying 12.2 - 12.3 VAC to the bu...,"[3, 4]",1.0,0
85261,This product does not work out of the box. I i...,"[0, 0]",1.0,0
85256,sadly i was not able to make this work - the ...,"[3, 3]",1.0,0
85254,I purchased this for my DeWalt dw715 miter saw...,"[3, 5]",1.0,0
85251,After being advertised as working with the Bos...,"[2, 3]",1.0,0
85239,Does not fit the Dewalt DW717 Mitre Saw! I cou...,"[1, 2]",1.0,0
85234,"It might work on other brands, but not on a De...","[6, 11]",1.0,0
85297,I bought this months ago but was unable to ins...,"[1, 2]",1.0,0
85230,I tried everything to get this to work but it ...,"[0, 0]",1.0,0


In [161]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [162]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [163]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(134476, 3)

In [151]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']].head(2)

In [78]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



In [79]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [164]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        #reconstruct sentence
        review[count] = " ".join(eachSent)
    #a clobbed together without pun
    wholeText = [" ".join(review) ]
    #reconstruct paragraph of sentences

    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

sample = "This is a test. It is only a test!"
paragraphText, wholeText = singleReviewCleanUp(sample)
wholeText, paragraphText
#paragraphText

(['this is a test it is only a test'], ['this is a test', 'it is only a test'])

In [165]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [166]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 3min 57s


In [23]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [84]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [85]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [87]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [563]:
#print(stopWords)

In [353]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [88]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [167]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 7min 37s


In [168]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 152 ms


In [93]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']].head(2)
#reviewTextProcessing.dtypes

In [169]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [170]:
reviewTextProcessing.shape
#reviewTextProcessing.columns

(134476, 7)

In [151]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [171]:
reviewTextProcessing['CleanedJoin'].head(2)

0    i hate it when my shirt collars not otherwise ...
1    these little magnets are really powerful for t...
Name: CleanedJoin, dtype: object

In [172]:

import itertools

largeText = reviewTextProcessing.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

134476

In [173]:
sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [174]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 10000


In [175]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [176]:

size = int(len(largeText)*.75)
x_train_text = largeText[:size]
x_test_text = largeText[size:]

y_train =  reviewTextProcessing['rating'][:size]
y_test = reviewTextProcessing['rating'][size:]
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [177]:
len(y_train), len(x_train_tokens), len(x_train_text), len(x_test_text)

(100857, 100857, 100857, 33619)

In [1072]:
#x_train_tokens[1], x_train_text[1]

In [178]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [179]:
np.mean(num_tokens), np.max(num_tokens)


(110.80553407299443, 4838)

In [180]:
np.std(num_tokens)

132.92503796578936

In [181]:
max_tokens = np.mean(num_tokens) + 1 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


243

In [182]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.897914869567804

In [183]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape
x_test_pad.shape


(33619, 243)

In [184]:
for i in range(0, 1):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    2 1558
    4   39   18 3329 8531   11  722 2917   12  236   82 1219  220   41
   12 2405  908 2048    1  225    2  212   68  424 2317  951    5   32
   17   37 1753   19   27  120   64 1190    2  996   41   96    8   37
 1753 

In [185]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [186]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [187]:
x_train_text[1]


'these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of a pound'

In [188]:
tokens_to_string(x_train_tokens[1])


'these little magnets are really powerful for there size i am using them to make secret compartments in custom made boxes each one about of a pound'

In [113]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [190]:
model = Sequential()
#embedding_size = 8 > 0.8694
embedding_size = 10

In [191]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [192]:
model.add(GRU(units=16, return_sequences=True))

In [193]:
model.add(GRU(units=8, return_sequences=True))


In [194]:
model.add(GRU(units=4))


In [195]:
model.add(Dense(1, activation='sigmoid'))


In [196]:
optimizer = Adam(lr=1e-3)


In [197]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [198]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 243, 10)           100000    
_________________________________________________________________
gru_6 (GRU)                  (None, 243, 16)           1296      
_________________________________________________________________
gru_7 (GRU)                  (None, 243, 8)            600       
_________________________________________________________________
gru_8 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 102,057
Trainable params: 102,057
Non-trainable params: 0
_________________________________________________________________


In [199]:
# %%time
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=5, batch_size=64)

model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95814 samples, validate on 5043 samples
Epoch 1/3
95814/95814 [==============================] - 424s 4ms/sample - loss: 0.1697 - acc: 0.9605 - val_loss: 0.1231 - val_acc: 0.9657
Epoch 2/3
95814/95814 [==============================] - 410s 4ms/sample - loss: 0.1186 - acc: 0.9635 - val_loss: 0.1128 - val_acc: 0.9653
Epoch 3/3
95814/95814 [==============================] - 395s 4ms/sample - loss: 0.0970 - acc: 0.9674 - val_loss: 0.1065 - val_acc: 0.9681


In [200]:
%%time
result = model.evaluate(x_test_pad, y_test)



33619/33619 [==============================] - 66s 2ms/sample - loss: 0.1018 - acc: 0.9673
Wall time: 1min 6s


In [201]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 96.73%


In [202]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

Wall time: 2.13 s


In [203]:
cls_pred = np.array([1.0 if p>0.2 else 0.0 for p in y_pred])


In [204]:
cls_true = np.array(y_test[0:1000])


In [205]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [209]:
len(incorrect)
cls_pred < 1

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [141]:
reduceData.loc[7695+40, ['reviewText', 'overall', 'rating']]
reviewTextProcessing.loc[7695+40, ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

CleanedJoin       so heres the thing that no one else wants to s...
overall                                                           1
rating                                                            0
SentimentModel                          [-0.44696969696969685, 0.0]
Name: 7735, dtype: object

In [207]:
idx = incorrect[1]
print(type(idx))
for each in incorrect[:10]:
    idx = each
    text = x_test_text[idx]
    print(idx, text, cls_pred[idx], cls_true[idx], " BlobText ",
         TextBlob(text).sentiment.polarity)


<class 'numpy.int64'>
10 it is rather limited in what it can cut, worst thng is that the depth is not adjustable, so many other options and i made a mistake and bought this 1.0 0  BlobText  -0.17410714285714285
158 these bulbs claim to be a very white light, i purchased these to replace the k fluorescents i had in my bathroom lights as the fluorescents were very slow to get bright, i had hoped that these would be decent at least as they are advertised as being very white light but there is no color temperature measurement anywhere unfortunately these bulbs are about k k extremely yellow or soft white just like every other incandescent bulb on the market, i am not sure what everyone s age old obsession with not being able to see properly in their house is but i definitely do not enjoy it, with this style of bulb however it seems i do not have a choice in that room 1.0 0  BlobText  0.03342592592592592
190 for the most part this machine is a good compromise that does a fairly good job but

In [1108]:
reviewTextProcessing.loc[list(incorrect + 7695), ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

,CleanedJoin,overall,rating,SentimentModel
7735,"so heres the thing that no one else wants to say, the thing is built like a piece of shit, it jingles and rattles and parts are always moving around, i used this times and it shorted out, the lights started flashing and it got hot, i called line and they determined it was bad so they opened a ticket, fyi if you want to deal with the worst customer service in the industry just call line not only do they not know what they were talking about but in the backgrounc i could hear them lauging and joking like there was a big party going on, i wish i would have never bought this but hopefully i get a new one soon, well see how helpful they really are",1.0,0,"[-0.44696969696969685, 0.0]"
7736,"i have always loved line this product sounded great for about an hour then total gave up, it went dead, i changed channels and it worked for like a minute then went dead, i am sorry to say that i returned it, my akg wireless guitar bug has lasted years with no drop outs i am really disappointed",1.0,0,"[-1.15, 0.0]"
7750,"i was very excited to try this pedal when it finally arrived, just to find out it wasn t working, it is defective and because i brought it to mexico now i can not get a refund or a replacement i will try to fix it through some local electrician or something, very dissapointing",1.0,0,"[0.44375000000000003, 1.0]"
7850,one of the ball bearings designed to catch and lock the side mounted to the strap dislodged and my very expensive fender custom shop telecaster fell flat on the ground,1.0,0,"[-0.3375, 0.0]"
7911,"sounded good, warmer sounding than stock tubes in my ac, i rocked all night and loved the sound, next day i turned it on my amp was crackling out and wiping out the signal i checked the tubes and one of them was making violent purple flashes before the signal burnt out, i then noticed that one of my fuses blew out as well, i changed out the fuse and checked again and the tube was still doing the same thing so i just swapped the tubes back for atock, id like to see a real set of jjs im sure one of them was a dud",1.0,0,"[1.2, 1.0]"
7992,i bought these just to add a layer of sound in my blues tunes i record the notes do not bend easily if you think your will cheat the big boys harps youll be dissapointed quickly these are little more than toys compared to dollar harps they feel cheap but have a decent sound i dont know if they will last but to make harp noises without decent bends ill use these sparingly jim,2.0,0,"[0.17604166666666665, 1.0]"
7996,"i play harp with some some that i perform, i found that all of these harmonica s the note s are off and it has it is hard to play i have g blues harp that i compared the g harp to and the notes were a little off, the c harp was terrible, that said the only thing i like about this order is the case, i going to see if i can return them for store credit",1.0,0,"[-1.2395833333333333, 0.0]"
8007,"too thick no flexibility, sounds really harsh and like it is ruining my ukulele strings, tried on bass too again with less than satisactory results",1.0,0,"[-0.6583333333333333, 0.0]"
8010,"the picks work well with bass but they are just too thick to use with ukulele, the pics also shed easily making your instrument dusty in no time",2.0,0,"[-0.20833333333333337, 0.0]"
8036,"i guess the tension on these is more than my uke was used to and when i put them on the neck became severely bowed after a day or two, sounded lovely for the first hour that has that is why two stars, granted my uke is one of the cheaper lanikais if you have a better built uke it might be able to take these but i simply can not recommend personally",2.0,0,"[1.125, 1.0]"


In [1]:
!pip install textblob


In [328]:

top_dict = {}
#for c in data_dtm.columns:
#    top = data_dtm[c].sort_values(ascending=False).head(30)
#    top_dict[c]= list(zip(top.index, top.values))

top_dict['cable'] = 3
top_dict['not happy'] = 4
top_dict['disappoint'] = 7

regeneration = ""

for key, value in top_dict.items():
    space = ((key) + " ")
    details = (space) * value
    regeneration = regeneration + details
    
review = " ".join(top_dict.keys())
top_dict.values()
regeneration



'cable cable cable not happy not happy not happy not happy disappoint disappoint disappoint disappoint disappoint disappoint disappoint '

In [271]:
from wordcloud import WordCloud

#stopwords=stopWords,
wc = WordCloud(background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [10]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [413]:
#handling money
#>>> a = re.findall(r"\$\d+.\d+", text2)

In [236]:
#numeric and fraction handling
#domain specific if applies to tool/measurement?
pattern = re.compile(r'^\d+/\d+$')

def evaluateFraction(data, pattern=pattern):
    try:
        re.match(pattern, data)
        return eval(data)  
    except:
        print(f"Not fraction related input {data}")
        return False      

In [266]:
pd.options.display.max_colwidth = 5000

#productsReviewData.cleanedText.head(2)

In [121]:
sampleData = productsReviewData